In [10]:
import pandas as pd
import numpy as np
import fxcmpy
import time
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use("seaborn")

from strategies.MACDBacktester import MACDBacktester as MACDTester

class MACDTrader():
    
    def __init__(self, instrument, bar_length, EMA_S, EMA_L, signal_mw, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units
        self.position = 0
        self.tc = 0.00007
        #*****************add strategy-specific attributes here******************
        self.EMA_S = EMA_S
        self.EMA_L = EMA_L
        self.signal_mw = signal_mw
        #************************************************************************        
        self.api = None
    def connect(self):
        self.api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

    def compute_optimal_parameters(self):
        df = self.raw_data.copy()
        df["Close"] = df[self.instrument]
        macdTester = MACDTester(self.instrument, df, self.EMA_S, self.EMA_L, self.signal_mw, self.tc)
        macdTester.optimize_parameters((5,20,1), (21,50,1), (5,20,1))
        print("MACD Test Results: ",macdTester.test_strategy())
        parameters =  macdTester.get_parameters()
        self.EMA_S = parameters[0]
        self.EMA_L = parameters[1]
        self.signal_mw = parameters[2]
        print("MACD parameters: ", parameters)
        self.EMA_S, self.EMA_L, self.signal_mw = macdTester.get_parameters()
                        
    def get_most_recent(self, period = "m5", number = 10000):
        while True:  
            time.sleep(10)
            df = self.api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                #self.compute_optimal_parameters()
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************

        df["EMA_S"] = df[self.instrument].ewm(span = self.EMA_S, min_periods = self.EMA_S).mean()            
        df["EMA_L"] = df[self.instrument].ewm(span = self.EMA_L, min_periods = self.EMA_L).mean()
        df["MACD"] = df.EMA_S - df.EMA_L            
        df["MACD_Signal"] = df.MACD.ewm(span = self.signal_mw, min_periods = self.signal_mw).mean()
        df["position"] = np.where(df["MACD"] > df["MACD_Signal"], 1, 0)
        df.dropna(inplace = True)
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        
            order_info = {}
            target_profit = 1.05
            stop_loss = 0.95            
            close = self.tick_data[self.instrument].iloc[-1]
            if len(order_info) > 0:
                target_profit_price = order_info["target_profit"]
                stop_loss_price = order_info["stop_loss"]
                buy_price = order_info["buy_price"]
             
                if close > target_profit_price:
                    print(f"SELL: Target Profit hit || bought_price: {buy_price}, target profit: {target_profit_price}")
                    order = self.api.create_market_sell_order(self.instrument, self.units)
                    self.report_trade(order, "GOING SHORT")  # go short with full amount
                    self.position = 0 # short position
                    order_info = {}
                elif close < stop_loss_price:
                    print(f"SELL: Stop Loss hit || bought_price: {buy_price}, stop loss: {stop_loss_price}")
                    order = self.api.create_market_buy_order(self.instrument, self.units)
                    self.report_trade(order, "GOING LONG") # go short with full amount
                    self.position = 0 # short position
                    order_info = {}
                           
            elif self.data["position"].iloc[-1] == 1: # signal to go long
                if self.position == 0:
                    order = self.api.create_market_buy_order(self.instrument, self.units)
                    self.report_trade(order, "GOING LONG")# go long with full amount
                    self.position = 1  # long position
                    
                    stop_loss_price = 0
                    target_profit_price = 0
                    
                    stop_loss_price = close * stop_loss
                    target_profit_price = close * target_profit
                    
                    order_info["buy_price"] = close
                    order_info["target_profit"] = target_profit_price
                    order_info["stop_loss"] = stop_loss_price
                    print(f"Stop loss: {stop_loss_price}")
                    print(f"Target: {target_profit_price}")
                    
            elif self.data["position"].iloc[-1] == 0: # signal to go short
                if self.position == 1:
                    order = self.api.create_market_sell_order(self.instrument, self.units)
                    self.report_trade(order, "GOING SHORT") 
                    self.position = 0 # short position
                    order_info = {}
                    

    def report_trade(self, order, going):
        time = order.get_time()
        units = self.api.get_open_positions().amountK.iloc[-1]
        price = self.api.get_open_positions().open.iloc[-1]
        unreal_pl = self.api.get_open_positions().grossPL.sum()
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | Unreal. P&L = {}".format(time, units, price, unreal_pl))
        print(100 * "-" + "\n")
    

In [11]:
instrument = "ETH/USD"
bar_size = "5min"
ema_s = 19
ema_l = 23
signal = 10

trader = MACDTrader(instrument, bar_size, ema_s, ema_l, signal, 1)

In [12]:
trader.api = fxcmpy.fxcmpy(config_file= "fxcm.cfg")

In [13]:
trader.api.is_connected()

True

In [14]:
trader.get_most_recent()

In [15]:
trader.api.subscribe_market_data(instrument, (trader.get_tick_data, ))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 
----------------------------------------------------------------------------------------------------
2021-11-11 20:30:03.790000 | GOING LONG
2021-11-11 20:30:03.790000 | units = 0.001 | price = 4750.15 | Unreal. P&L = -65.05286
----------------------------------------------------------------------------------------------------

Stop loss: 4510.5525
Target: 4985.3475
27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 
----------------------------------------------------------------------------------------------------
2021-11-11 20:35:13.685000 | GOING SHORT
2021-11-11 20:35:13.685000 | units = 0.001 | price = 4743.15 | Unreal. P&L = -67.26944
----------------------------------------------------------------------------------------------------

68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 10

Exception in thread Thread-198599:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.8/3.8.11/Frameworks/Python.framework/Versions/3.8/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python@3.8/3.8.11/Frameworks/Python.framework/Versions/3.8/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/site-packages/socketio/client.py", line 616, in _handle_eio_message
    self._handle_event(pkt.namespace, pkt.id, pkt.data)
  File "/usr/local/lib/python3.8/site-packages/socketio/client.py", line 500, in _handle_event
    r = self._trigger_event(data[0], namespace, *data[1:])
  File "/usr/local/lib/python3.8/site-packages/socketio/client.py", line 550, in _trigger_event
    return self.handlers[namespace][event](*args)
  File "/usr/local/lib/python3.8/site-packages/fxcmpy/fxcmpy.py", line 2555, in __on_price_update__
    callbacks[func](data, self.prices

950 951 952 953 954 955 956 957 958 959 960 961 962 963 964 965 966 967 968 969 970 971 972 973 974 975 976 977 978 979 980 981 982 983 984 985 986 987 988 989 990 991 992 993 994 995 996 997 998 999 1000 1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1017 1018 1019 1020 1021 1022 1023 1024 1025 1026 1027 1028 

In [ ]:
#api.subscribe_market_data(instrument, (trader.get_tick_data, ))

starttime = time.time()
timeout = time.time() + 60*60*6
while time.time() <= timeout:
    time.sleep(900 - ((time.time() - starttime) % 900.0))
    api.unsubscribe_market_data(instrument)


In [ ]:
trader.api.unsubscribe_market_data(instrument)


In [9]:
trader.api.close()